In [7]:
import sys
sys.path.append("../python")

import json
import h5py
import numpy as np
import arviz as az
import compute_costs

In [8]:
parameters_path_mlda = "../configs/mlda_with_orientation.json"
    
file_mlda = open(parameters_path_mlda)
config_mlda = json.load(file_mlda)
file_mlda.close()

# load samples
chains_mlda = config_mlda["Setup"]["Chains"]
samples_mlda = []
for c in range(chains_mlda):
    sample_path = config_mlda["Setup"]["OutputPath"] + config_mlda["Sampling"]["ResultFile"] 
    fin = h5py.File(sample_path + "_" + str(c) + ".h5")
    samples_mlda.append(np.array(fin['/samples']))
    fin.close()

In [9]:
parameters_path_mh = "../configs/mh_with_orientation.json"
    
file_mh = open(parameters_path_mh)
config_mh = json.load(file_mh)
file_mh.close()

# load samples
chains_mh = config_mh["Setup"]["Chains"]
samples_mh = []
for c in range(chains_mh):
    sample_path = config_mh["Setup"]["OutputPath"] + config_mh["Sampling"]["ResultFile"] 
    fin = h5py.File(sample_path + "_" + str(c) + ".h5")
    samples_mh.append(np.array(fin['/samples'][:,250000:]))
    fin.close()

In [10]:
# compute ess
ess_mh = np.zeros((chains_mh,3))
ess_1_mh = np.zeros((chains_mh,3))
ess_2_mh = np.zeros((chains_mh,3))
for c in range(chains_mh):
    ess_mh[c][0] = az.ess(samples_mh[c][0])
    ess_mh[c][1] = az.ess(samples_mh[c][1])
    ess_mh[c][2] = az.ess(samples_mh[c][2])

ess_mlda = np.zeros((chains_mlda,3))
ess_1_mlda = np.zeros((chains_mlda,3))
ess_2_mlda = np.zeros((chains_mlda,3))
for c in range(chains_mlda):
    ess_mlda[c][0] = az.ess(samples_mlda[c][0])
    ess_mlda[c][1] = az.ess(samples_mlda[c][1])
    ess_mlda[c][2] = az.ess(samples_mlda[c][2])

In [11]:
print("Samples per effective sample \n")

num_samples_mh = config_mh["Sampling"]["NumSamples"] - config_mh["Sampling"]["BurnIn"]
print("MCMC (" + str(num_samples_mh) + ")")
for c in range(chains_mh):
    print("Sample " + str(c) + ": " + str(num_samples_mh/min(ess_mh[c][0],ess_mh[c][1],ess_mh[c][2])))

num_samples_mlda = config_mlda["Sampling"]["NumSamples"] - config_mlda["Sampling"]["BurnIn"]
print("MLDA (" + str(num_samples_mlda) + ")")
for c in range(chains_mlda):
    print("Sample " + str(c) + ": " + str(num_samples_mlda/min(ess_mlda[c][0],ess_mlda[c][1],ess_mlda[c][2])))

Samples per effective sample 

MCMC (750000)
Sample 0: 49891.09742519653
Sample 1: 795.4519413739401
Sample 2: 435.06844938789607
Sample 3: 3313.4488245834355
Sample 4: 300.1478691675277
MLDA (30000)
Sample 0: 18449.97811818519
Sample 1: 255.04322795859505
Sample 2: 117.6110408288201
Sample 3: 562.7928377686874
Sample 4: 179.17927669024778


In [14]:
print("Costs per effective sample \n")

print("MCMC")
costs_mh = compute_costs.compute_costs(parameters_path_mh)
print(costs_mh)
num_samples_mh = config_mh["Sampling"]["NumSamples"]
for c in range(chains_mh):
    print("Sample " + str(c) + ": " + str(costs_mh*num_samples_mh/min(ess_mh[c][0],ess_mh[c][1],ess_mh[c][2])))

print("MLDA")
costs_mlda = compute_costs.compute_costs(parameters_path_mlda)
print(costs_mlda)
num_samples_mlda = config_mlda["Sampling"]["NumSamples"]
for c in range(chains_mlda):
    print("Sample " + str(c) + ": " + str(costs_mlda*num_samples_mlda/min(ess_mlda[c][0],ess_mlda[c][1],ess_mlda[c][2])))

Costs per effective sample 

MCMC (1000000)
Sample 0: 66521.46323359538
Sample 1: 1060.6025884985868
Sample 2: 580.0912658505281
Sample 3: 4417.931766111247
Sample 4: 400.1971588900369
MLDA (30000)
Sample 0: 147599.82494548152
Sample 1: 2040.3458236687604
Sample 2: 940.8883266305608
Sample 3: 4502.342702149499
Sample 4: 1433.4342135219822
